In [ ]:
# import libraries
import pandas as pd
import re
from datetime import datetime
from pathlib import Path

# load data
profiles = pd.read_csv('../csv/19.01.2026/profiles.csv')
predictions = pd.read_csv('../csv/10.02.2026/predictions_rows.csv')
fixtures = pd.read_csv('../csv/10.02.2026/fixtures_rows(1).csv')

# Merge predictions with profiles on user_id
merged = predictions.merge(profiles, left_on='user_id', right_on='id', how='left', suffixes=('_pred', '_profile'))

# Merge with fixtures on fixture_id
merged = merged.merge(fixtures, left_on='fixture_id', right_on='id', how='left', suffixes=('', '_fixture'))
df_shape = merged[["username", "club", "home_team", "away_team", "home_prediction", "away_prediction", "home_score", "away_score", "game_week_id"]]

# remove inactive players
df_players = df_shape.drop(df_shape[df_shape["username"] == "Martinez"].index)

# remove non premier league weeks
df_game_weeks = df_players.drop(df_players[df_players["game_week_id"].isin(["7d6130ac-980a-4352-8ec0-29a06643525e", "ef99d1e9-75a3-4c2a-9ab3-48604f86b4fa"])].index)

# convert all numbers to integers
cols_to_convert = ["home_score", "away_score", "home_prediction", "away_prediction"]

df_game_weeks[cols_to_convert] = df_game_weeks[cols_to_convert].astype(int)

df_final = df_game_weeks

# check the final dataframe
print (df_final.shape)
print (df_final.head())
print (df_final.dtypes)

In [ ]:
# clean up team names

team_fixes = {
    # Brighton
    "Brighton": "Brighton & Hove Albion",
    "Brighton and Hove Albion": "Brighton & Hove Albion",

    # Bournemouth
    "Bournemouth": "AFC Bournemouth",

    # Wolves
    "Wolves": "Wolverhampton Wanderers",

    # Man United
    "Man Utd": "Manchester United",
    "Man U": "Manchester United",
    "Manchester Utd": "Manchester United",
    "Manchester United ": "Manchester United",

    # Man City
    "Man City": "Manchester City",

    # Aston Villa
    "Aston Villa ": "Aston Villa",

    # West Ham
    "West Ham": "West Ham United",
    " West Ham": "West Ham United",

    # Crystal Palace
    " Crystal Palace": "Crystal Palace",

    # Brentford
    " Brentford": "Brentford",

    # Leeds
    "Leeds": "Leeds United",
    " Leeds": "Leeds United",

    # Newcastle
    "Newcastle": "Newcastle United",

    # Nottingham Forest
    "Nottm Forest": "Nottingham Forest",

    # Tottenham
    "Tottenham": "Tottenham Hotspur",
    " Tottenham Hotspur": "Tottenham Hotspur",

    # Chelsea
    " Chelsea": "Chelsea"
}

# apply fixes
df_final["home_team"] = df_final["home_team"].replace(team_fixes)
df_final["away_team"] = df_final["away_team"].replace(team_fixes)

# show unique values
print("Home")
print(df_final["home_team"].nunique())
print(sorted(df_final["home_team"].unique()))

print("\nAway")
print(df_final["away_team"].nunique())
print(sorted(df_final["away_team"].unique()))

In [ ]:
print(df_final.head())
print("home team counts")
print(df_final["home_team"].value_counts())
print("away team counts")
print(df_final["away_team"].value_counts())
print("game week counts")
print(df_final["game_week_id"].value_counts())
print(df_final["game_week_id"].nunique())

In [ ]:
# Create predicted league table for The Barman
player_name = '🍺The Barman'
player_data = df_final[df_final['username'] == player_name].copy()

# Get all unique teams
all_teams = pd.concat([player_data['home_team'], player_data['away_team']]).unique()
print(f"Found {len(all_teams)} teams")

# Initialize league table dictionary
league_table = []

for team in all_teams:
    # Get all matches for this team
    home_matches = player_data[player_data['home_team'] == team]
    away_matches = player_data[player_data['away_team'] == team]
    
    # Calculate stats
    wins = 0
    draws = 0
    losses = 0
    goals_for = 0
    goals_against = 0
    
    # Home matches
    for _, match in home_matches.iterrows():
        home_goals = match['home_prediction']
        away_goals = match['away_prediction']
        goals_for += home_goals
        goals_against += away_goals
        
        if home_goals > away_goals:
            wins += 1
        elif home_goals == away_goals:
            draws += 1
        else:
            losses += 1
    
    # Away matches
    for _, match in away_matches.iterrows():
        home_goals = match['home_prediction']
        away_goals = match['away_prediction']
        goals_for += away_goals
        goals_against += home_goals
        
        if away_goals > home_goals:
            wins += 1
        elif away_goals == home_goals:
            draws += 1
        else:
            losses += 1
    
    # Calculate points and goal difference
    points = (wins * 3) + (draws * 1)
    goal_diff = goals_for - goals_against
    
    league_table.append({
        'Team': team,
        'Wins': wins,
        'Draws': draws,
        'Losses': losses,
        'Goals For': goals_for,
        'Goals Against': goals_against,
        'Goal Difference': goal_diff,
        'Points': points
    })

# Create DataFrame and sort
league_df = pd.DataFrame(league_table)
league_df = league_df.sort_values(
    by=['Points', 'Goal Difference', 'Goals For', 'Team'],
    ascending=[False, False, False, True]
).reset_index(drop=True)

# Add position index (1-20)
league_df.index = range(1, len(league_df) + 1)

# Get player's favorite club
fav_club = player_data['club'].iloc[0] if len(player_data) > 0 else ''

print(f"\n{player_name} - {fav_club}")
print("="*80)
print(league_df.to_string())

In [ ]:
def create_predicted_league_table(player_name, df_final):
    """Create a predicted league table for a specific player based on their predictions"""
    player_data = df_final[df_final['username'] == player_name].copy()
    
    if len(player_data) == 0:
        print(f"No data found for {player_name}")
        return None
    
    # Get all unique teams
    all_teams = pd.concat([player_data['home_team'], player_data['away_team']]).unique()
    
    # Initialize league table dictionary
    league_table = []
    
    for team in all_teams:
        # Get all matches for this team
        home_matches = player_data[player_data['home_team'] == team]
        away_matches = player_data[player_data['away_team'] == team]
        
        # Calculate stats
        wins = 0
        draws = 0
        losses = 0
        goals_for = 0
        goals_against = 0
        
        # Home matches
        for _, match in home_matches.iterrows():
            home_goals = match['home_prediction']
            away_goals = match['away_prediction']
            goals_for += home_goals
            goals_against += away_goals
            
            if home_goals > away_goals:
                wins += 1
            elif home_goals == away_goals:
                draws += 1
            else:
                losses += 1
        
        # Away matches
        for _, match in away_matches.iterrows():
            home_goals = match['home_prediction']
            away_goals = match['away_prediction']
            goals_for += away_goals
            goals_against += home_goals
            
            if away_goals > home_goals:
                wins += 1
            elif away_goals == home_goals:
                draws += 1
            else:
                losses += 1
        
        # Calculate points and goal difference
        points = (wins * 3) + (draws * 1)
        goal_diff = goals_for - goals_against
        
        league_table.append({
            'Team': team,
            'Wins': wins,
            'Draws': draws,
            'Losses': losses,
            'Goals For': goals_for,
            'Goals Against': goals_against,
            'Goal Difference': goal_diff,
            'Points': points
        })
    
    # Create DataFrame and sort
    league_df = pd.DataFrame(league_table)
    league_df = league_df.sort_values(
        by=['Points', 'Goal Difference', 'Goals For', 'Team'],
        ascending=[False, False, False, True]
    ).reset_index(drop=True)
    
    # Add position index (1-20)
    league_df.index = range(1, len(league_df) + 1)
    
    return league_df

# Test with The Barman
player_name = '🍺The Barman'
player_data = df_final[df_final['username'] == player_name]
fav_club = player_data['club'].iloc[0] if len(player_data) > 0 else ''

table = create_predicted_league_table(player_name, df_final)

print(f"\n{player_name} - {fav_club}")
print("="*100)
print(table.to_string())
print(f"\nTotal predictions used: {len(player_data)}")

In [ ]:
# Generate tables for all players
try:
    import dataframe_image as dfi
except ImportError as exc:
    raise ImportError("Missing package: dataframe_image. Install with pip install dataframe_image") from exc

def sanitize_filename_component(value):
    text = '' if pd.isna(value) else str(value)
    text = text.strip().replace(' ', '_')
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'[^A-Za-z0-9_-]+', '', text)
    return text

date_stamp = datetime.now().strftime('%d%m%y')
output_dir = Path('..') / 'Output' / f'players_tables_{date_stamp}'
output_dir.mkdir(parents=True, exist_ok=True)

all_players = df_final['username'].unique()
print(f"Generating predicted league tables for {len(all_players)} players...\n")

for player in sorted(all_players):
    # Get player's favorite club
    player_data = df_final[df_final['username'] == player]
    fav_club = player_data['club'].iloc[0] if len(player_data) > 0 else ''
    
    # Create table
    table = create_predicted_league_table(player, df_final)
    
    if table is not None:
        print(f"\n{'='*100}")
        print(f"{player} - {fav_club}")
        print('='*100)
        
        # Style the table
        styled_table = table.style.set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#4472C4'), ('color', 'white'), 
                                         ('border', '1px solid black'), ('font-weight', 'bold'), 
                                         ('text-align', 'center')]},
            {'selector': 'td', 'props': [('border', '1px solid black'), ('padding', '8px'), 
                                         ('text-align', 'center'), ('color', '#000000')]},
            {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#F2F2F2')]},
            {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#FFFFFF')]},
            {'selector': 'tr:first-child', 'props': [('background-color', '#90EE90')]},
        ])
        
        safe_player = sanitize_filename_component(player) or 'UnknownPlayer'
        safe_club = sanitize_filename_component(fav_club) or 'UnknownClub'
        output_path = output_dir / f'{safe_player}_{safe_club}.png'
        dfi.export(styled_table, output_path, table_conversion='matplotlib')
        
        display(styled_table)
        print(f"\nPredictions used: {len(player_data)} matches")
        print(f"Saved PNG: {output_path}")
        print("\n")

print(f"\nAll {len(all_players)} tables generated!")

In [ ]:
# Average predicted league table across all players
avg_tables = []
for player in sorted(all_players):
    table = create_predicted_league_table(player, df_final)
    if table is not None:
        t = table.reset_index(drop=False).rename(columns={'index': 'Position'})
        t['Player'] = player
        avg_tables.append(t)

avg_df = pd.concat(avg_tables, ignore_index=True)

numeric_cols = ['Wins', 'Draws', 'Losses', 'Goals For', 'Goals Against', 'Goal Difference', 'Points']
avg_df[numeric_cols] = avg_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

avg_table = (
    avg_df.groupby('Team', as_index=False)[numeric_cols]
    .mean()
    .sort_values(by=['Points', 'Goal Difference', 'Goals For', 'Team'], ascending=[False, False, False, True])
    .reset_index(drop=True)
)

avg_table[numeric_cols] = avg_table[numeric_cols].round(0).astype(int)

avg_table.index = range(1, len(avg_table) + 1)

styled_avg = avg_table.style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#4472C4'), ('color', 'white'),
                                 ('border', '1px solid black'), ('font-weight', 'bold'),
                                 ('text-align', 'center')]},
    {'selector': 'td', 'props': [('border', '1px solid black'), ('padding', '8px'),
                                 ('text-align', 'center'), ('color', '#000000')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#F2F2F2')]},
    {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#FFFFFF')]},
    {'selector': 'tr:first-child', 'props': [('background-color', '#90EE90')]},
])

avg_output_path = output_dir / 'averarge.png'
dfi.export(styled_avg, avg_output_path, table_conversion='matplotlib')

display(styled_avg)
print(f"\nSaved PNG: {avg_output_path}")
